In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from imblearn.over_sampling import SMOTE

In [138]:
df_original = pd.read_excel('Data/EWS_data_total.xlsx')

In [139]:
#Calendarization
#split KalJahr/Monat into Month and Year
df_calendarization = df_original.copy()
df_calendarization[['Month', 'Year']] = df_calendarization['KalJahr/Monat'].astype(str).str.split('.', expand=True).astype(int)

# Group by Product_Category, Year, Month and sum Units
monthly_totals  = df_calendarization.groupby(['Product_Category', 'Year', 'Month'])['Units'].sum().reset_index(name='Units_Month')

available_months = monthly_totals.groupby(['Product_Category', 'Year'])['Month'].nunique().reset_index(name='Num_Months')

monthly_totals = monthly_totals.merge(available_months, on=['Product_Category', 'Year'], how='left')

#Compute partial-year total units
partial_annual_totals = monthly_totals.groupby(['Product_Category', 'Year'])['Units_Month'].sum().reset_index(name='Units_Year')

#Calculate total units per Product_Category and Year
#annual_totals = monthly_totals.groupby(['Product_Category', 'Year'])['Units_Month'].sum().reset_index(name='Units_Year')

#Merge and compute monthly calendarization factor
calendarization_factors = monthly_totals.merge(partial_annual_totals, on=['Product_Category', 'Year'])
calendarization_factors['Calendarization_Factor'] = calendarization_factors['Units_Month'] / calendarization_factors['Units_Year']

calendarization_factors_final = calendarization_factors[['Product_Category', 'Year', 'Month', 'Calendarization_Factor']] #520

#calendarization_factors_final.to_excel("calendarization_factors.xlsx", index=False)
calendarization_factors_final

,Product_Category,Year,Month,Calendarization_Factor
0,Product_Category_AA,2022,1,0.083203
1,Product_Category_AA,2022,2,0.076780
2,Product_Category_AA,2022,3,0.092856
3,Product_Category_AA,2022,4,0.079186
4,Product_Category_AA,2022,5,0.077438
...,...,...,...,...
515,Product_Category_BH,2024,12,0.090293
516,Product_Category_BH,2025,1,0.285491
517,Product_Category_BH,2025,2,0.198128
518,Product_Category_BH,2025,3,0.191888


In [140]:
#create a dim tbl with product info
product_info = df_original[['Product_ID', 'Product_Line', 'Product_Category']].drop_duplicates()
product_info['Product_ID'] = 'Product_' + product_info['Product_ID'].astype(str)  #1.478

In [141]:
product_info.to_excel('product_info.xlsx')

In [142]:
df = df_original

In [143]:
df['Customer_ID'] = 'Customer_' + df['Kunde_ID'].astype(str)
df['Product_ID'] = 'Product_' + df['Product_ID'].astype(str)
df['Year'] = df['KalJahr/Monat'].astype(str).str[-4:]
df['Month'] = df['KalJahr/Monat'].astype(str).str.split('.').str[0].astype(int)

# Convert to datetime for proper grouping
df['YearMonth'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01')
df      #894.077 rows

,KalJahr/Monat,Units,Kunde_ID,Product_ID,Product_Line,Product_Category,Customer_ID,Year,Month,YearMonth
0,2.2022,30,22,Product_674,Product_Line_A,Product_Category_AB,Customer_22,2022,2,2022-02-01
1,3.2022,90,22,Product_674,Product_Line_A,Product_Category_AB,Customer_22,2022,3,2022-03-01
2,5.2022,60,22,Product_674,Product_Line_A,Product_Category_AB,Customer_22,2022,5,2022-05-01
3,1.2022,10,444,Product_543,Product_Line_B,Product_Category_BA,Customer_444,2022,1,2022-01-01
4,4.2022,10,444,Product_543,Product_Line_B,Product_Category_BA,Customer_444,2022,4,2022-04-01
...,...,...,...,...,...,...,...,...,...,...
894072,4.2025,375,443,Product_872,Product_Line_B,Product_Category_BF,Customer_443,2025,4,2025-04-01
894073,4.2025,750,443,Product_801,Product_Line_B,Product_Category_BF,Customer_443,2025,4,2025-04-01
894074,4.2025,150,443,Product_809,Product_Line_B,Product_Category_BF,Customer_443,2025,4,2025-04-01
894075,4.2025,60,443,Product_1196,Product_Line_B,Product_Category_BF,Customer_443,2025,4,2025-04-01


In [144]:
# Calculate units per product &  year month 
df_product_grouping = df.groupby(['YearMonth', 'Product_ID'])['Units'].sum().reset_index()  #49.129
df_product_grouping  

,YearMonth,Product_ID,Units
0,2022-01-01,Product_1,9930
1,2022-01-01,Product_10,2560
2,2022-01-01,Product_100,2180
3,2022-01-01,Product_1001,3150
4,2022-01-01,Product_1002,17
...,...,...,...
49124,2025-04-01,Product_995,2070
49125,2025-04-01,Product_996,260
49126,2025-04-01,Product_997,1080
49127,2025-04-01,Product_998,4450


In [145]:
#merge with product info
df_product_grouping = df_product_grouping.merge(product_info, on='Product_ID', how= 'left')
df_product_grouping['Year'] = df_product_grouping['YearMonth'].dt.year
df_product_grouping['Month'] = df_product_grouping['YearMonth'].dt.month
df_product_grouping

,YearMonth,Product_ID,Units,Product_Line,Product_Category,Year,Month
0,2022-01-01,Product_1,9930,Product_Line_A,Product_Category_AA,2022,1
1,2022-01-01,Product_10,2560,Product_Line_A,Product_Category_AA,2022,1
2,2022-01-01,Product_100,2180,Product_Line_A,Product_Category_AB,2022,1
3,2022-01-01,Product_1001,3150,Product_Line_A,Product_Category_AB,2022,1
4,2022-01-01,Product_1002,17,Product_Line_A,Product_Category_AE,2022,1
...,...,...,...,...,...,...,...
49124,2025-04-01,Product_995,2070,Product_Line_B,Product_Category_BF,2025,4
49125,2025-04-01,Product_996,260,Product_Line_A,Product_Category_AB,2025,4
49126,2025-04-01,Product_997,1080,Product_Line_B,Product_Category_BF,2025,4
49127,2025-04-01,Product_998,4450,Product_Line_A,Product_Category_AC,2025,4


In [146]:
#merge calendarization factors 
df_calendarization_merged = df_product_grouping.merge(calendarization_factors_final, on=['Product_Category', 'Year', 'Month'], how='left')

In [147]:
#calculate unique customer count per product per month
df_customer_grouping = df.groupby(['YearMonth', 'Product_ID'])['Customer_ID'].nunique().reset_index()
df_customer_grouping.rename(columns={'Customer_ID': 'Unique_Customers'}, inplace=True)

In [148]:
#merge product and customer grouping on YearMonth and Product_ID
df_combined = pd.merge(
    df_calendarization_merged,
    df_customer_grouping,
    on=['YearMonth', 'Product_ID'],
    how='left')  #49.129

In [149]:
df = pd.DataFrame(df_combined)

In [150]:
# Sort for proper lag/rolling calc
df.sort_values(['Product_ID', 'YearMonth'], inplace=True)
df.rename(columns={'Units': 'Units_sold'}, inplace=True)

#Normalization to annualized units
df['Units_annualized'] = df['Units_sold'] / df['Calendarization_Factor'] #annualizing the values—standardizing monthly data as if every month were equally long and comparable >> eliminates calendar-induced bias

In [151]:
#product level relative normalization
df['Units_rel_to_avg'] = df.groupby(['Product_ID', 'Year'])['Units_annualized'].transform(lambda x: x / x.mean())
#>>scales the calendar-adjusted values relative to their own product-year avg, giving you a dimensionless index:
#1.0 = avg month
#>1.0 = better-than-avg (for this product & year)
#<1.0 = weaker month
#keep only Units_norm as transactional data
#Units_norm>> removes seasonality/calendar bias. Monthly fluctuations caused by different month lengths or holidays are neutralized. It allows consistent comparison across time

In [152]:
# Drop other units-related columns
df_cleaned = df.drop(columns=['Units_annualized', 'Calendarization_Factor'])

In [153]:
df = df_cleaned

In [154]:
df['Units_CM'] = df['Units_rel_to_avg']

In [155]:
#units & customers last month
df['Units_LM'] = df.groupby('Product_ID')['Units_CM'].shift(1)
df['Customers_LM'] = df.groupby('Product_ID')['Unique_Customers'].shift(1)

#convert NaN to 0
df['Customers_LM'] = df['Customers_LM'].fillna(0)
df['Units_LM'] = df['Units_LM'].fillna(0)
df.drop(columns=['Units_rel_to_avg'], inplace= True)

In [156]:
#6-month units trend (slope of a linear fit) >> returns the direction and steepness of the recent 6-month trend.
def calc_slope(x):
    if len(x) < 4:
        return np.nan
    return np.polyfit(range(4), x[-4:], 1)[0]  # slope of last 6 values >> performs a linear regression on the last 6 data points
#range(6) → represents time steps: [0, 1, 2, 3, 4, 5]
#x[-6:] → takes the last 6 Units_sold values for the product
#[0] → extracts the slope (first coefficient of the polynomial)

# Calculate 6-month rolling mean of units
df['Rolling_mean_4m'] = (
    df.groupby('Product_ID')['Units_CM']
    .rolling(4).mean()
    .reset_index(level=0, drop=True))

df['trend_4m'] = df.groupby('Product_ID')['Units_CM']\
    .transform(lambda x: x.rolling(window=4, min_periods=4).apply(calc_slope, raw=False))

# Convert absolute slope to % slope
df['trend_4m_pct'] = df['trend_4m'] / df['Rolling_mean_4m']

#groupby('Product_ID') → ensures the trend is calculated per product
#rolling(window=6) → applies the calculation over each 6-month window
#transform(...) → keeps the result aligned with the original DataFrame
#result: Units_trend_6m
#Is > 0 → product sales are trending up over the last 6 months
#Is < 0 → product sales are declining
#Is 0 → stable sales
#Is NaN for the first 5 rows per product (not enough history)

In [157]:
#MoM % growth
df['MoM_Growth_pct'] = (df['Units_CM'] - df['Units_LM']) / df['Units_LM']

# Flag first month per product
df['Is_First_Month'] = (df.groupby('Product_ID')['YearMonth'].rank(method='first') == 1).astype(int)

#df['MoM_Growth_pct'] = np.clip(df['MoM_Growth_pct'], a_min=None, a_max=1e6)  # capping inf values
df.loc[df['Units_LM'] == 0, 'MoM_Growth_pct'] = 0  #replace first month growth to 0 >> fitlering put for modeling

In [158]:
df['MoM_Growth_pct_Cust'] = (df['Unique_Customers'] - df['Customers_LM']) / df['Customers_LM']
df.loc[df['Customers_LM'] == 0, 'MoM_Growth_pct_Cust'] = 0 

Metrics to trend

In [159]:
#Rolling trend over 3 months (slope of last 3 months)
#trend calculation using linear regression
def calc_trend_4m(x):
    if len(x) < 4:
        return np.nan
    y = x.values.reshape(-1, 1)
    X = np.arange(len(x)).reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    return model.coef_[0][0]  # slope

#trend units
df['Trend_4M'] = (
    df.groupby('Product_ID')['Units_CM']
    .rolling(window=4, min_periods=4)
    .apply(calc_trend_4m, raw=False)
    .reset_index(level=0, drop=True))

#trend customers
df['Trend_4M_C'] = (
    df.groupby('Product_ID')['Unique_Customers']
    .rolling(window=4, min_periods=4)
    .apply(calc_trend_4m, raw=False)
    .reset_index(level=0, drop=True))

#vs trend in abs
def current_vs_trend(x):
    if len(x) < 4:
        return np.nan
    y = x.values.reshape(-1, 1)
    X = np.arange(len(x)).reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)
    return y[-1][0] - y_pred[-1][0]  # Actual - Predicted for last point

#vs trend in pct
def current_vs_trend_pct(x):
    if len(x) < 4:
        return np.nan
    y = x.values.reshape(-1, 1)
    X = np.arange(len(x)).reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)
    return (y[-1][0] - y_pred[-1][0]) / y_pred[-1][0]  # % deviation

# Apply per product
df['vs_trend_abs'] = (
    df.groupby('Product_ID')['Units_CM']
    .rolling(window=4, min_periods=4)
    .apply(current_vs_trend, raw=False)
    .reset_index(level=0, drop=True))

df['vs_trend_pct_Prod'] = (
    df.groupby('Product_ID')['Units_CM']
    .rolling(window=4, min_periods=4)
    .apply(current_vs_trend_pct, raw=False)
    .reset_index(level=0, drop=True))

df['vs_trend_pct_Cust'] = (
    df.groupby('Product_ID')['Unique_Customers']
    .rolling(window=4, min_periods=4)
    .apply(current_vs_trend_pct, raw=False)
    .reset_index(level=0, drop=True))

Volatility

In [160]:
#Rolling std dev (volatility)
df['Rolling_std_4m'] = (df.groupby('Product_ID')['Units_CM'].rolling(4).std().reset_index(level=0, drop=True))

#Coefficient of variation >> CV = σ / μ (standard deviation over mean)
#It reflects relative variability of the sales in the 3-month context
df['Volatility'] = df['Rolling_std_4m'] / df['Rolling_mean_4m']

Consecutive decline

In [161]:
#consecutive decline flag over 3 months
def consecutive_decline(series):
    drops = (series.diff() < 0).astype(int) # series.diff() computes the difference between CM and the LM. flags it as a drop if CM is lower than LM.
    return drops.rolling(window=3, min_periods=3).sum().apply(lambda x: 1 if x == 3 else 0)

df['Consec_Drop_Prod'] = df.groupby('Product_ID')['Units_CM'].apply(lambda x: consecutive_decline(x)).reset_index(level=0, drop=True)

CONSEQ CUSTOMER DROP

In [162]:
def consecutive_decline_customers(series):
    drops = (series.diff() < 0).astype(int)
    return drops.rolling(window=3, min_periods=3).sum().apply(lambda x: 1 if x == 3 else 0)

df['Consec_Drop_Cust'] = df.groupby('Product_ID')['Unique_Customers'].apply(consecutive_decline_customers).reset_index(level=0, drop=True)
df['Target_ConsecCustDrop'] = df['Consec_Drop_Cust']

AVG UNITS PER CUSOMER

In [163]:
#metric for shrinking order sizes despite stable customer count.
df['Units_per_Cust'] = df['Units_CM'] / df['Unique_Customers']
# Rolling mean or trend over 4 months
df['Trend_Units_per_Cust'] = (
    df.groupby('Product_ID')['Units_per_Cust']
    .rolling(window=4, min_periods=4)
    .apply(calc_trend_4m, raw=False)
    .reset_index(level=0, drop=True))

# Flag
df['Target_UnitsPerCust_Drop'] = (df['Trend_Units_per_Cust'] < 0).astype(int)

Negative trend over last 3 months

In [ ]:
df['NegTrend'] = (df['trend_4m'] < 0).astype(int)

#Add a persistence score: tracks how many months in a row the trend has been negative:
def count_negative_trends(series):
    count = 0
    counts = []
    for val in series:
        if val < 0:
            count += 1
        else:
            count = 0
        counts.append(count)
    return counts

df['Trend_Negative_Streak'] = df.groupby('Product_ID')['trend_4m'].transform(count_negative_trends) # represents how many months in a row a product has had a negative trend.
#df['NegTrend_Scaled'] = df['Trend_Negative_Streak'].clip(upper=4) / 4   #If any value is greater than 4, just treat it as 4. we don't want very high streaks to dominate the score. Four months of continuous decline is already severe.

Normalization to peak

In [165]:
#Normalization current month vs 75th percentile. This avoids overemphasizing outliers that might occur in one extreme month.
#Compute 75th percentile of units per Product_ID
df['P75_Units'] = df.groupby('Product_ID')['Units_CM'].transform(lambda x: x.quantile(0.75))   # calculates the 75th percentile (Q3) for each Product_ID

# Normalize current units sold by 75th percentile value
df['CM_vs_P75'] = df['Units_CM'] / df['P75_Units']

In [166]:
df['Target_UnderTrend_Cust'] = (df['vs_trend_pct_Cust'] < -0.35).astype(int)  
df['Target_Volatility'] = (df['Volatility']> 0.3).astype(int)
df['Target_UnderTrend'] = (df['vs_trend_pct_Prod']< -0.3).astype(int)
df['Target_ConsecDrop'] = (df['Consec_Drop_Prod'] == 1).astype(int)
df['Target_NegTrend'] = (df['Trend_Negative_Streak'] > 2).astype(int)
df['Target_vs_P75'] = (df['CM_vs_P75'] < 0.3).astype(int) 

In [ ]:
df['Target_Next_Month_Risk_Label'] = (
    df[['Target_UnderTrend_Cust', 'Target_Volatility', 'Target_UnderTrend', 'Target_ConsecDrop', 'Target_NegTrend']]
    .sum(axis=1)
    .apply(lambda x: 2 if x >= 4 else (1 if x >= 2 else 0)))

,YearMonth,Product_ID,Units_sold,Product_Line,Product_Category,Year,Month,Unique_Customers,Units_CM,Units_LM,...,Trend_Negative_Streak,P75_Units,CM_vs_P75,Target_UnderTrend_Cust,Target_Volatility,Target_UnderTrend,Target_ConsecDrop,Target_NegTrend,Target_vs_P75,Target_Next_Month_Risk_Label
0,2022-01-01,Product_1,9930,Product_Line_A,Product_Category_AA,2022,1,74,0.953081,0.000000,...,0,1.14673,0.831129,0,0,0,0,0,0,0
1249,2022-02-01,Product_1,10890,Product_Line_A,Product_Category_AA,2022,2,69,1.132671,0.953081,...,0,1.14673,0.987740,0,0,0,0,0,0,0
2499,2022-03-01,Product_1,8820,Product_Line_A,Product_Category_AA,2022,3,69,0.758542,1.132671,...,0,1.14673,0.661483,0,0,0,0,0,0,0
3752,2022-04-01,Product_1,10650,Product_Line_A,Product_Category_AA,2022,4,69,1.074050,0.758542,...,1,1.14673,0.936620,0,0,0,0,0,0,0
4999,2022-05-01,Product_1,10080,Product_Line_A,Product_Category_AA,2022,5,75,1.039511,1.074050,...,0,1.14673,0.906500,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44411,2024-12-01,Product_999,510,Product_Line_A,Product_Category_AB,2024,12,6,1.470495,0.524599,...,2,1.11864,1.314538,0,1,0,0,0,0,0
45590,2025-01-01,Product_999,630,Product_Line_A,Product_Category_AB,2025,1,6,1.146655,1.470495,...,0,1.11864,1.025044,0,1,0,0,0,0,0
46783,2025-02-01,Product_999,240,Product_Line_A,Product_Category_AB,2025,2,6,0.451386,1.146655,...,1,1.11864,0.403513,0,1,1,0,0,0,1
47965,2025-03-01,Product_999,750,Product_Line_A,Product_Category_AB,2025,3,7,1.292657,0.451386,...,2,1.11864,1.155561,0,1,0,0,0,0,0


In [169]:
# Shift all targets by -1 month to predict 1 month ahead
#! last row will now have NaN in the target >> should be dropped during training ?
df['Target_UnderTrend_shifted'] = df['Target_UnderTrend'].shift(-1)
df['Target_UnderTrend_Cust_shifted'] = df['Target_UnderTrend_Cust'].shift(-1)
df['Target_Volatility_shifted'] = df['Target_Volatility'].shift(-1)
df['Target_ConsecDrop_shifted'] = df['Target_ConsecDrop'].shift(-1)
df['Target_vs_P75_shifted'] = df['Target_vs_P75'].shift(-1)  
df['Target_NegTrend_shifted'] = df['Target_NegTrend'].shift(-1)
df['Target_Next_Month_Risk_shifted'] = df['Target_Next_Month_Risk_Label'].shift(-1)  #49.129

CLEANING

In [ ]:
"""# Drop rows with NaN targets (usually last month)
df_model = df.dropna(subset=[
    'Target_UnderTrend_shifted',
    'Target_MoMDecline_shifted',
    'Target_Volatility_shifted',
    'Target_ConsecDrop_shifted',
    'Target_vs_P75_shifted']) """

"# Drop rows with NaN targets (usually last month)\ndf_model = df.dropna(subset=[\n    'Target_UnderTrend_shifted',\n    'Target_MoMDecline_shifted',\n    'Target_Volatility_shifted',\n    'Target_ConsecDrop_shifted',\n    'Target_vs_P75_shifted']) "

In [170]:
#drop & export first months
first_months = df[df['Is_First_Month'] == 1]   #1.478
first_months.to_excel("dropped_first_months.xlsx") #for later to append to the main dataset
df = df[df['Is_First_Month'] == 0]    #47.651
df_model_sorted = df.sort_values(by= ['Product_ID', 'YearMonth'])   #47.651

In [ ]:
""" #drop & export last months
last_months = df_model_sorted.groupby('Product_ID')['YearMonth'].transform('max') 
df_dropped_last_months = df_model_sorted[df_model_sorted['YearMonth']==last_months]  #1.467 >> for later to append to the main dataset
df_dropped_last_months.to_excel("dropped_last_months.xlsx")
df_filtered = df_model_sorted[df_model_sorted['YearMonth'] < last_months].copy() #46.184 """

' #drop & export last months\nlast_months = df_model_sorted.groupby(\'Product_ID\')[\'YearMonth\'].transform(\'max\') \ndf_dropped_last_months = df_model_sorted[df_model_sorted[\'YearMonth\']==last_months]  #1.467 >> for later to append to the main dataset\ndf_dropped_last_months.to_excel("dropped_last_months.xlsx")\ndf_filtered = df_model_sorted[df_model_sorted[\'YearMonth\'] < last_months].copy() #46.184 '

In [171]:
df_filtered = df_model_sorted

In [172]:
# Replace all inf/-inf values with NaN & fill NaNs with 0 
df_filtered.replace([np.inf, -np.inf], np.nan, inplace=True)
df_filtered.fillna(0, inplace=True)
df_model = df_filtered

In [173]:
df_filtered.to_excel("df_preprocessed.xlsx")

In [174]:
#Dim Product
dim_product = pd.read_excel("product_info.xlsx")

In [ ]:
dim_product_transformed = dim_product.copy()

for col in dim_product.columns:
    if dim_product[col].dtype == 'object':
        dim_product_transformed[col + '_short'] = dim_product[col].apply(
            lambda x: x.split('_')[-1] if isinstance(x, str) else x)

dim_product_transformed = dim_product_transformed.drop(columns='Unnamed: 0')
dim_product_transformed.to_excel('dim_product.xlsx')